In [ ]:
from time import time
import numpy as np
import nibabel as nib
from dipy.core.gradients import gradient_table
from dipy.data import get_data
import dipy.reconst.NODDIx as NODDIx
from scipy.linalg import get_blas_funcs

In [ ]:
fname, fscanner = get_data('NODDIx_example')
params = np.loadtxt(fscanner)
img = nib.load(fname)
data = img.get_data()
affine = img.affine

In [ ]:
bvecs = params[:, 0:3]
G = params[:, 3] / 10 ** 6  # gradient strength
big_delta = params[:, 4]
small_delta = params[:, 5]
gamma = 2.675987 * 10 ** 8
bvals = gamma ** 2 * G ** 2 * small_delta ** 2 * (big_delta - small_delta / 3.)
gtab = gradient_table(bvals, bvecs, big_delta=big_delta,
                      small_delta=small_delta,
                      b0_threshold=0, atol=1e-2)

In [ ]:
def norm_meas_HCP(ydatam, b):

    """
    calculates std of the b0 measurements and normalizes all ydatam
    """
    b1 = np.where(b > 1e-5)
    b2 = range(b.shape[0])
    C = np.setdiff1d(b2, b1)
    b_zero_all = ydatam[C]
    b_zero_norm = sum(b_zero_all) / C.shape[0]
    y = ydatam / b_zero_norm
    b_zero_all1 = y[C]
    return y

In [ ]:
fit_method = 'MIX'
NODDIx_model = NODDIx.NODDIxModel(gtab, params,
                                  fit_method=fit_method)
NODDIx_fit = np.zeros((data.shape[0], data.shape[1], data.shape[2], 11))

In [ ]:
t1 = time()
for i in range(1,2):  # range(data.shape[0]):
    for j in range(1):  # range(data.shape[1]):
        for k in range(data.shape[2]):  # in range(1):
#            if mask[i, j, k] > 0:
            signal = np.array(data[i, j, k])
            signal = norm_meas_HCP(signal, bvals)
#            signal = np.float64(signal)
#               signal_n = add_noise(signal, snr=20, noise_type='rician')
            NODDIx_fit[i, j, k, :] = NODDIx_model.fit(signal)
            print(i)
t2 = time()
fast_time = t2 - t1
print(fast_time)

In [ ]:
affine = img.affine.copy()
nib.save(nib.Nifti1Image(NODDIx_fit[:, :, :, 0], affine),
         'f11_HCP_NODDIx.nii.gz')
nib.save(nib.Nifti1Image(NODDIx_fit[:, :, :, 1], affine),
         'f21_HCP_NODDIx.nii.gz')
nib.save(nib.Nifti1Image(NODDIx_fit[:, :, :, 2], affine),
         'f12_HCP_NODDIx.nii.gz')
nib.save(nib.Nifti1Image(NODDIx_fit[:, :, :, 3], affine),
         'f22_HCP_NODDIx.nii.gz')
nib.save(nib.Nifti1Image(NODDIx_fit[:, :, :, 4], affine),
         'f3_HCP_NODDIx.nii.gz')
nib.save(nib.Nifti1Image(NODDIx_fit[:, :, :, 5], affine),
         'OD1_HCP_NODDIx.nii.gz')
nib.save(nib.Nifti1Image(NODDIx_fit[:, :, :, 6], affine),
         'theta1_HCP_NODDIx.nii.gz')
nib.save(nib.Nifti1Image(NODDIx_fit[:, :, :, 7], affine),
         'phi1_HCP_NODDIx.nii.gz')
nib.save(nib.Nifti1Image(NODDIx_fit[:, :, :, 8], affine),
         'OD2_HCP_NODDIx.nii.gz')
nib.save(nib.Nifti1Image(NODDIx_fit[:, :, :, 9], affine),
         'theta2_HCP_NODDIx.nii.gz')
nib.save(nib.Nifti1Image(NODDIx_fit[:, :, :, 10], affine),
         'phi2_HCP_NODDIx.nii.gz')